<a href="https://colab.research.google.com/github/prerna-roy/spam_detection_nlp/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import regex as re 
import numpy as np

In [3]:
df_all = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DL Project - 6 9 22/data/emailspam_clean_to_from.csv")
df_all.head()

In [9]:
df = df_all[~df_all['message'].str.contains("Content-Type")]

In [10]:
print("Size of dataset without content-type rows =", len(df))
print("Size of original dataset", len(df_all))
df.drop(columns="Unnamed: 0")

Size of dataset without content-type rows = 40046
Size of original dataset 72798


In [16]:
df.head()

,label,subject,message,Recipient,Sender
1,0,Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",debian-mirrors@lists.debian.org,yan.morin@savoirfairelinux.com
3,1,Nice talking with ya,"Hey Billy, \n\nit was really fun going out the...",opt4@speedy.uwaterloo.ca,vqucsmdfgvsg@ruraltek.com
9,0,[R] Confidence-Intervals.... help...,Hi...\n\nI have to use R to find out the 90% c...,r-help@stat.math.ethz.ch,jjfahr@ucalgary.ca
16,1,hi man,"Hey Billy, \n\nit was really fun going out the...","the00@speedy.uwaterloo.ca', 'opt2@speedy.uwate...",slqwil@planetonline.com
19,0,Re: [R] Confidence-Intervals.... help...,Hm... sounds like a homework problem to me...\...,jjfahr@ucalgary.ca,sarah.goslee@gmail.com


In [17]:
# Replace all newline characters with a space
df['message'] = df['message'].str.strip().str.replace("\n", ' ')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,label,subject,message,Recipient,Sender
1,0,Typo in /debian/README,"Hi, i've just updated from the gulus and I che...",debian-mirrors@lists.debian.org,yan.morin@savoirfairelinux.com
3,1,Nice talking with ya,"Hey Billy, it was really fun going out the o...",opt4@speedy.uwaterloo.ca,vqucsmdfgvsg@ruraltek.com
9,0,[R] Confidence-Intervals.... help...,Hi... I have to use R to find out the 90% con...,r-help@stat.math.ethz.ch,jjfahr@ucalgary.ca
16,1,hi man,"Hey Billy, it was really fun going out the o...","the00@speedy.uwaterloo.ca', 'opt2@speedy.uwate...",slqwil@planetonline.com
19,0,Re: [R] Confidence-Intervals.... help...,Hm... sounds like a homework problem to me... ...,jjfahr@ucalgary.ca,sarah.goslee@gmail.com


In [25]:
#Check double newlines have been changed too
df[df['message'].str.contains("\n\n")]

,label,subject,message,Recipient,Sender


In [26]:
#Check there are no more messages with newline characters
df[df['message'].str.contains("\n")]

,label,subject,message,Recipient,Sender


In [ ]:
#Get length of each message and store in a new column 

In [44]:
df.groupby('label').describe().T

label                                                             0  \
subject   count                                               21110   
          unique                                              11724   
          top                                      CNN Alerts: bush   
          freq                                                  318   
message   count                                               21110   
          unique                                              20573   
          top     ______________________________________________...   
          freq                                                    8   
Recipient count                                               20990   
          unique                                               2521   
          top                              r-help@stat.math.ethz.ch   
          freq                                                 2403   
Sender    count                                               21109   
          unique                                               2727   
          top                                      tridge@samba.org   
          freq                                                  685   

label                                                             1  
subject   count                                               18936  
          unique                                               8518  
          top                           Avis Important et Personnel  
          freq                                                 2974  
message   count                                               18936  
          unique                                              11938  
          top     Cher(e) membre                  Desjardins/ Ac...  
          freq                                                 2606  
Recipient count                                               18823  
          unique                                               3271  
          top                             theorize@plg.uwaterloo.ca  
          freq                                                 1963  
Sender    count                                               18932  
          unique                                              12938  
          top                 services.de.cartes@scd.desjardins.com  
          freq                                                 2826

In [55]:
# The top message for the not spam messages seems to be many underscores, check to see what these messages look like 
df[ (df['message'].str.contains(" _" and "_ "))].tail(1)

,label,subject,message,Recipient,Sender
72796,0,Re: [R] RODBC problem,"Hello, as I wrote I call sqlFetch(channel,...",r-help@stat.math.ethz.ch,Bernhard.Wellhoefer@gaia-group.com


In [52]:
df.label.value_counts()

0    21110
1    18936
Name: label, dtype: int64

In [54]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [63]:
# Test out what tokens are made using a message that contains links/email addresses 
# Just using one message for testing, this is NOT for the full dataset
token2 = word_tokenize(df.iloc[2].message)
print(token2)

['Hi', '...', 'I', 'have', 'to', 'use', 'R', 'to', 'find', 'out', 'the', '90', '%', 'confidence-interval', 'for', 'the', 'sensitivity', 'and', 'specificity', 'of', 'the', 'following', 'diagnostic', 'test', ':', 'A', 'particular', 'diagnostic', 'test', 'for', 'multiple', 'sclerosis', 'was', 'conducted', 'on', '20', 'MS', 'patients', 'and', '20', 'healthy', 'subjects', ',', '6', 'MS', 'patients', 'were', 'classified', 'as', 'healthy', 'and', '8', 'healthy', 'subjects', 'were', 'classified', 'as', 'suffering', 'from', 'the', 'MS', '.', 'Furthermore', ',', 'I', 'need', 'to', 'find', 'the', 'number', 'of', 'MS', 'patients', 'required', 'for', 'a', 'sensitivity', 'of', '1', '%', '...', 'Is', 'there', 'a', 'simple', 'R-command', 'which', 'can', 'do', 'that', 'for', 'me', '?', 'I', 'am', 'completely', 'new', 'to', 'R', '...', 'Help', 'please', '!', 'Jochen', '--', 'View', 'this', 'message', 'in', 'context', ':', 'http', ':', '//www.nabble.com/Confidence-Intervals', '....', '-help', '...', '-tf

In [64]:
token2 = [token.lower() for token in token2] #convert to all lowercase words
print(token2)

['hi', '...', 'i', 'have', 'to', 'use', 'r', 'to', 'find', 'out', 'the', '90', '%', 'confidence-interval', 'for', 'the', 'sensitivity', 'and', 'specificity', 'of', 'the', 'following', 'diagnostic', 'test', ':', 'a', 'particular', 'diagnostic', 'test', 'for', 'multiple', 'sclerosis', 'was', 'conducted', 'on', '20', 'ms', 'patients', 'and', '20', 'healthy', 'subjects', ',', '6', 'ms', 'patients', 'were', 'classified', 'as', 'healthy', 'and', '8', 'healthy', 'subjects', 'were', 'classified', 'as', 'suffering', 'from', 'the', 'ms', '.', 'furthermore', ',', 'i', 'need', 'to', 'find', 'the', 'number', 'of', 'ms', 'patients', 'required', 'for', 'a', 'sensitivity', 'of', '1', '%', '...', 'is', 'there', 'a', 'simple', 'r-command', 'which', 'can', 'do', 'that', 'for', 'me', '?', 'i', 'am', 'completely', 'new', 'to', 'r', '...', 'help', 'please', '!', 'jochen', '--', 'view', 'this', 'message', 'in', 'context', ':', 'http', ':', '//www.nabble.com/confidence-intervals', '....', '-help', '...', '-tf

In [68]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
stop = stopwords.words('english')
token2 = [token for token in token2 if token not in stop]
print(token2)

['hi', '...', 'use', 'r', 'find', '90', '%', 'confidence-interval', 'sensitivity', 'specificity', 'following', 'diagnostic', 'test', ':', 'particular', 'diagnostic', 'test', 'multiple', 'sclerosis', 'conducted', '20', 'ms', 'patients', '20', 'healthy', 'subjects', ',', '6', 'ms', 'patients', 'classified', 'healthy', '8', 'healthy', 'subjects', 'classified', 'suffering', 'ms', '.', 'furthermore', ',', 'need', 'find', 'number', 'ms', 'patients', 'required', 'sensitivity', '1', '%', '...', 'simple', 'r-command', '?', 'completely', 'new', 'r', '...', 'help', 'please', '!', 'jochen', '--', 'view', 'message', 'context', ':', 'http', ':', '//www.nabble.com/confidence-intervals', '....', '-help', '...', '-tf3544217.html', '#', 'a9894014', 'sent', 'r', 'help', 'mailing', 'list', 'archive', 'nabble.com', '.', '______________________________________________', 'r-help', '@', 'stat.math.ethz.ch', 'mailing', 'list', 'https', ':', '//stat.ethz.ch/mailman/listinfo/r-help', 'please', 'read', 'posting',

In [ ]:
[token for token in token2 if token not in ascii.charlist]

## Data Cleaning procedure 
1. Remove blank rows/rows with N/A
2. change all text to lower case 
3. remove all messages with 'content-type' if desired (though tokenization should still work even with it)
4. Tokenize all messages 
5. Remove stop words and non-numeric characters (non-numeric characters is optional to remove, could be useful as some of these messages have links/emails inside of them)
6. Perform word lemmatization 
7. Create a new column in the original dataframe for `clean_text` which has undergone all the preprocessing 
8. [Optional]: Create a new column `meaningful_text` which contains only words that are longer than 2 letters (but this may remove punctuation/abbreviations etc that could potentially be useful) 